> Note: Open notebook with aaq_template environment as the Kernel.

In [ ]:
!pip install requests

In [ ]:
import requests
import os

## Import environment variables

Make sure the following is placed into the conda env file below: 

`[conda install path]/miniconda3/envs/aaq_template/etc/conda/activate.d/env_vars.sh`

In [ ]:
# Bearer token for authentication
INBOUND_CHECK_TOKEN = os.getenv("INBOUND_CHECK_TOKEN")

# Hosts
local_host = os.getenv("LOCAL_HOST")        # mc_core local
dev_host = os.getenv("DEV_HOST")            # mc_core dev
staging_host = os.getenv("STAGING_HOST")    # mc_core staging

Set host to use for notebook:

In [ ]:
host = staging_host
bearer_token = "Bearer " + INBOUND_CHECK_TOKEN

# inbound/check
Variables from this cell will be used for the other two endpoints

In [ ]:
# Test the check endpoint
response = requests.post(
    host + "/inbound/check",
    json={"text_to_match": "test message"},
    headers={"Authorization": bearer_token},
)

# store info from response
top_response = response.json()["top_responses"][0][0]
# for feedback endpoint
inbound_id = response.json()["inbound_id"]
feedback_secret_key = response.json()["feedback_secret_key"]
# for pagination endpoint (also inbound_id above)
inbound_secret_key = response.json()["inbound_secret_key"]
initial_next_page_url = response.json()["next_page_url"]

# print results
print("top response:", top_response)
print("check endpoint response time (s):", response.elapsed.total_seconds())

In [ ]:
# response.json()

# inbound/feedback

In [ ]:
# Simple function to test the feedback endpoint
def feedback_endpoint_test(inbound_id, feedback_secret_key, feedback):

    feedback_response = requests.put(
        host + "/inbound/feedback",
        json={
            "inbound_id": inbound_id,
            "feedback_secret_key": feedback_secret_key,
            "feedback": feedback,
        },
        headers={"Authorization": bearer_token},
    )

    print("Feedback call response:", feedback_response.text)
    print("Feedback endpoint response time (s):", response.elapsed.total_seconds())

In [ ]:
# correct information
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0}, #or "page_number": 0. See below.
)

In [ ]:
# correct information
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "page_number": 0},
)

In [ ]:
# incorrect feedback format
feedback_endpoint_test(
    inbound_id=inbound_id, feedback_secret_key=feedback_secret_key, feedback="Gold Star"
)

In [ ]:
# incorrect inbound_id
feedback_endpoint_test(
    inbound_id=0,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0}, # or "page_number": 0
)

In [ ]:
# incorrect feedback_secret_key
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key="abcdefg",
    feedback={"feedback_type": "positive", "faq_id": 0}, # or "page_number": 0
)

# Test pagination 
#### inbound/inbound_id/page_number

There are two approaches here:
1. Jumping to a page using the page number
2. Using the next and previous page urls returned by each call to flip through pages

## Jump-to-page

In [ ]:
# Simple function to jump to a page and return content
def jump_to_page(page_number, inbound_secret_key):

    # call
    response = requests.get(
        url=host + "/inbound/" + inbound_id + "/" + str(page_number),
        params={
            "inbound_secret_key": inbound_secret_key
        },  #!# params, NOT json. Opposite of other endpoint. #!#
        headers={"Authorization": bearer_token},
    )

    # show results
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
    else:
        if response.json()["top_responses"] == []:
            print("No content.")
        else:
            print("top response:", response.json()["top_responses"][0][0])
        print("check endpoint response time (s):", response.elapsed.total_seconds())
        if "prev_page_url" in response.json():
            print("Previous page url:", "found")
        if "next_page_url" in response.json():
            print("Next page:", "found")

### Page 1 should match initial call response

In [ ]:
jump_to_page(1, inbound_secret_key)

### Page 2 should be different

In [ ]:
jump_to_page(2, inbound_secret_key)

### Go to non-existing page

In [ ]:
jump_to_page(-1, inbound_secret_key)

In [ ]:
jump_to_page(0, inbound_secret_key)

In [ ]:
jump_to_page(999, inbound_secret_key)

## Flip pages (next and previous)

### Next page from initial call should be the same as page 2

In [ ]:
# Test pagination endpoint - flip pages
response = requests.get(
    url=host + initial_next_page_url,
    headers={"Authorization": bearer_token},
)

# show results
print("top response:", response.json()["top_responses"][0][0])
print("check endpoint response time (s):", response.elapsed.total_seconds())

### Flip through all available pages

In [ ]:
url = initial_next_page_url
page_number = 2

while True:

    # print current page number
    print("Page", page_number)

    #### call
    response = requests.get(
        url=host + url,
        headers={"Authorization": bearer_token},
    )

    #### print results
    print("top response:", response.json()["top_responses"][0][0])
    print("check endpoint response time (s):", response.elapsed.total_seconds())

    #### return next/prev page link
    # Note: The response will simply not include the next/prev page url field if it
    # doesn't exist, resulting in an error if we try to select it from the JSON response.
    if "next_page_url" in response.json():
        url = response.json()["next_page_url"]
        page_number += 1
    else:
        print("ERROR: No next/prev page.")
        break